In [1]:
import pandas as pd
import json

# load CMMLU test data
market_data_path = "CMMLU/data/test/marketing.csv"
econ_data_path = "CMMLU/data/test/economics.csv"
ch_his_path = "CMMLU/data/test/chinese_history.csv"
# Read the CSV file
market_data = pd.read_csv(market_data_path)
econ_data = pd.read_csv(econ_data_path)
ch_his_data = pd.read_csv(ch_his_path)

test_data = ch_his_data
columns_as_lists = {col: test_data[col].tolist() for col in test_data.columns}

# Display the first few rows of the dataframe
#test_data.head(10)

In [2]:
# forumulate questions
initial_instruction = """
这个题目是单选题

#回答格式#
答案是【你的答案】来标注你的答案。

#答案选项#
A

B

C

D

"""

def initial_prompt(num):
    question = columns_as_lists['Question'][num]
    op_A = columns_as_lists['A'][num]
    op_B = columns_as_lists['B'][num]
    op_C = columns_as_lists['C'][num]
    op_D = columns_as_lists['D'][num]
    complete_prompt = f"{initial_instruction} \n {question}\n A {op_A} \n B {op_B} \n C {op_C} \n D {op_D}"
    question_only = f"{question}"
    options = f"A {op_A} \n B {op_B} \n C {op_C} \n D {op_D}"
    return complete_prompt, question_only, options

In [8]:
# remove the answer
df_test = test_data.iloc[:, :-1]

second_column_name = df_test.columns[0]
df_test = df_test.drop(columns=[second_column_name])

In [ ]:
#format the initial prompt
for row in df_test.iterrows():
    print(row[1])

In [3]:
import qianfan

QIANFAN_AK = "vtPwTlnPvgIdb4DRSuyNDuIN"
QIANFAN_SK = "Sp8CCvNsQgUsRKlJozQwWVClRliZ2S8m"

chat_comp = qianfan.ChatCompletion(ak=QIANFAN_AK, sk=QIANFAN_SK)

In [4]:
import re
# check if there is a pattern exist in the input string
# return the found pattern string else return None
def check_pattern(string, pattern):
    # Define the pattern
    pattern = pattern
    # Search for the pattern in the string
    match = re.search(pattern, string)
    # Return True if the pattern is found, otherwise False
    return match.group(0) if match else None

In [5]:
answers = columns_as_lists['Answer']
# this function test the if the agent answer the question correctly
def test(response, num):
    true_answer = answers[num]
    print(f"文言一心的回答是：{response}")
    print(f"Answer is {true_answer}")
    
    answer_part = check_pattern(response, r'答案.*[ABCD]') #find the string that contains only the answer
    
    if answer_part != None:
        letter = answer_part[-1]
        if letter==true_answer:
            print(f"question {num} is correct!")
            return 1
    else:    
        print("Error: didn't find answer for this question")
        return 0
    return 0

In [6]:
#Prompt 评分专家
prompt_analyst="""
你是一个优秀的Prompt专家，对于一个Prompt，你会按照这5个维度来打分。

#评分标准#

明确性（Clarity）：30分。如果Prompt不清晰或容易引起混淆，那么AI的回答可能会偏离预期，因此明确性是最基本的要求之一。

实用性（Practicality）：25分。Prompt的目的是要为用户提供实用的信息和解决问题的策略，所以实用性也是相当重要的。

创新性（Innovation）：15分。虽然这不是必需的，但创新性能够让Prompt产生独特的、富有洞察力的回答，有时甚至可能开辟全新的应用领域。

结果稳定性（Consistency of Output）：15分。为了确保用户可以依赖AI的回答，结果的一致性和预见性是非常重要的。

通用性（Universality）：15分。虽然有些Prompt可能特定于某一场景，但如果一个Prompt可以在多个场景下产生有效的结果，那么它的价值就更大。

#任务#
请给我的这个prompt打分，加总，并说明理由，最后做一个优化修改。

#工作流程#
1."分析你的问题（Prompt）",
2."基于评分标准，对提供的prompt进行各项评分",
3."对每一个评分项进行解释",
4."基于你的问题进行写出工作流程，回复不低于5个步骤",
5. Initialization，内容一定要是基于我提问的问题
5."生成回复，确保回复符合预期"
6.“提供一个优化修改之后的prompt”

#输出格式#
输出一个JSON，4个字段："提示词"，"优化后的提示词"，"总评分"，"工作流程"
{
    "提示词": ##【填入你要评估的提示词】##,
    "优化后的提示词"：##【在这里填入你优化后产生的提示词】##
    "总评分": #【你对这个prompt的总评分】#,
}

## Initialization：
    接下来我会给出我的问题(Prompt)，请根据我的Prompt
    1.基于[CRISPE提示框架]，请一步一步进行输出，直到最终输出[优化Promot]；
    2.输出完毕之后，请咨询我是否有需要改进的意见，如果有建议，请结合建议重新基于[CRISPE提示框架]输出。
    要求：请避免讨论[CRISPE提示框架]里的内容；
"""

In [7]:
import template_new
from template_new import *
prompt_optimize_prompt = simple_prompt

In [8]:
print(simple_prompt)

 
## 角色：Prompt工程师

## 任务：请你优化以下prompt

## 输出格式：JSON
{
    "优化之后的prompt"：<你优化之后的prompt>
}

## 你要优化的prompt




In [20]:
def prompt_optimization(initial_prompt, options):
    if len(initial_prompt) <= 1:
        raise ValueError("Invalid prompt input")
    optimized_prompt = " "
    input_prompt = CRISPE_prompt_multiple_choice + initial_prompt

    new_prompt = chat_comp.do(
            model="ERNIE-Bot", 
            messages=[{
                "role": "user",
                "content": input_prompt
                }],
            temperature = 0.00001)
    return new_prompt.body["result"]

In [21]:
p = prompt_optimization("""
Question    长征期间，国内外形势发生重大变化，民族矛盾加剧，中国共产党的方针政策进行重大调整。这一调整是指 
A                                   召开八七会议，确定武装反抗国民党反动派的总方针
B                                         召开七大，确立毛泽东思想的指导地位
C                                      发表“八一宣言”，号召停止内战，一致抗日
D                                         召开遵义会议，纠正“左”倾错误路线"""
,"""A                                   召开八七会议，确定武装反抗国民党反动派的总方针
B                                         召开七大，确立毛泽东思想的指导地位
C                                      发表“八一宣言”，号召停止内战，一致抗日
D                                         召开遵义会议，纠正“左”倾错误路线""")
print(p)

我准备好了，接下来我将基于您给出的Prompt，按照[CRISPE提示框架]进行逐步分析和优化，并输出最终的优化Prompt。

### Step 1: Context（上下文）
首先，我们需要明确问题的背景和上下文。这个问题是关于中国共产党在长征期间，面对国内外形势的重大变化，特别是民族矛盾的加剧，所做出的方针政策调整。

### Step 2: Role（角色）
在这个问题中，最适合扮演的角色是历史学者或党史专家，他们对中国共产党的历史，特别是长征时期的历史有深入的了解和研究。

### Step 3: Instruction（指令）
我们需要构建一个清晰的指令，引导LLM准确理解问题并给出答案。指令应该明确指出需要选择的是哪个时期的方针政策调整，并列出所有可能的选项供选择。

### Step 4: Prompt（提示）
根据[CRISPE提示框架]，我们可以将原始的Prompt优化为：

```json
{
    "提示词": "长征期间，中国共产党面对国内外形势的重大变化和民族矛盾的加剧，进行了方针政策的重大调整。请从以下选项中，选择出正确描述这一调整的内容。",
    "优化后的提示词": "在长征时期，中国共产党针对国内外形势的急剧变化和民族矛盾的显著加剧，进行了重要的方针政策调整。请仔细分析以下选项，并选出最符合这一历史事实的方针政策调整。",
    "选项": [
        "A. 召开八七会议，确定武装反抗国民党反动派的总方针（注：此会议发生在长征前，不符合题目时间范围）",
        "B. 召开七大，确立毛泽东思想的指导地位（注：七大召开于抗战胜利前夕，时间不符）",
        "C. 发表‘八一宣言’，号召停止内战，一致抗日（符合长征期间民族矛盾加剧的背景，是正确选项）",
        "D. 召开遵义会议，纠正‘左’倾错误路线（虽然重要，但并非针对民族矛盾加剧的直接调整）"
    ]
}
```

### Step 5: Explanation（解释）
在优化后的Prompt中，我增加了对每个选项的简短注释，以帮助理解每个选项与问题背景的关联性。特别是，我指出了A和B选项的时间不符，以及D选项虽然重要但与问题直接关联度不高的情况。

### Step 6: Workflow（工作流程）
1. **理解问题**

In [12]:
def grammer_checking_agent(json_input):
    prompt = """
    # # Role:JSON工程师
    1. Don't break character under any circumstance.
    2. Don't talk nonsense and make up facts.
    
    ## Skill:
    1. 熟悉[JSON框架]。
    2. 能够识别我提供的JSON里的问题。
    3. 能够修改JSON中的bug，使我的JSON输入完整而且没有bug
    
    ## Constrains:
    1. 不要增加输入的JSON的内容
    2. 不要增加输入的JSON的内容
    3. 与<OutputFormat>结构保持一致
    
    ## WorkFlow
    1. 识别输入JSON中的问题
    2. 修复输入JSON中的问题
    3. 根据OutputFormat输出修复完整的JSON
    
    ## OutputFormat
    输出一个JSON，3个字段："提示词"，"优化后的提示词"，"选项"
    {
        "提示词": ##【填入你要优化的提示词】##,
        "优化后的提示词":##【在这里填入你优化后产生的提示词】##,
        "选项":["A.#【选项A的内容】#", "B.#【选项B的内容】#", "C.#【选项C的内容】#", "D.#【选项D的内容】#"]
    }

    #输出示例#
    {
        "提示词": "你要优化的提示词是：下列注重卖方需要的市场营销管理哲学是",
        "优化后的提示词":"请选择一个注重卖方需要的市场营销管理哲学？您可以从市场营销观念、社会市场营销观念、客户观念和生产观念中挑选一个最符合您需求的选项。"
        "选项": ["A. 市场营销观念", "B. 社会市场营销观念", "C. 客户观念", "D. 生产观念"]
    }
    
    """
    response = llm(prompt + "你要修改的JSON是： " + json_input)
    
    return response

In [13]:
def check_json_format(input_json):
    try: 
        json.loads(input_json)
        return True
    except ValueError:
        return False

In [14]:
def fix_json(input_json):
    found_pattern = regex.compile(r'\{(?:[^{}]|(?R))*\}')
    json_str = input_json
    while (check_json_format(json_str) == False):
        json_str = grammer_checking_agent(json_str)
        json_str = found_pattern.findall(json_str)[0]
    return json_str

In [26]:
import regex
#处理优化后的提示词
def prompt_preprocess(prompt):
    found_pattern = regex.compile(r'\{(?:[^{}]|(?R))*\}')
    json_prompt = found_pattern.findall(prompt)
    if len(json_prompt) < 1:
        return prompt
    print(json_prompt[0])
    if check_json_format(json_prompt[0]):
        dict_prompt = json.loads(json_prompt[0]) # a dictionary contain the optimized prompt
        if "优化后的提示词" in dict_prompt:
            return dict_prompt["优化后的提示词"]
    else:
        print("incorrect json")
        correct_json_str = fix_json(json_prompt[0]) # use LLM to correct the grammer of JSON
        correct_json = found_pattern.findall(correct_json_str)
        dict_prompt = json.loads(correct_json[0])
        if "优化后的提示词" in dict_prompt:
            return dict_prompt["优化后的提示词"]
        return dict_prompt

In [27]:
prompt_preprocess(p)

{
    "提示词": "长征期间，中国共产党面对国内外形势的重大变化和民族矛盾的加剧，进行了方针政策的重大调整。请从以下选项中，选择出正确描述这一调整的内容。",
    "优化后的提示词": "在长征时期，中国共产党针对国内外形势的急剧变化和民族矛盾的显著加剧，进行了重要的方针政策调整。请仔细分析以下选项，并选出最符合这一历史事实的方针政策调整。",
    "选项": [
        "A. 召开八七会议，确定武装反抗国民党反动派的总方针（注：此会议发生在长征前，不符合题目时间范围）",
        "B. 召开七大，确立毛泽东思想的指导地位（注：七大召开于抗战胜利前夕，时间不符）",
        "C. 发表‘八一宣言’，号召停止内战，一致抗日（符合长征期间民族矛盾加剧的背景，是正确选项）",
        "D. 召开遵义会议，纠正‘左’倾错误路线（虽然重要，但并非针对民族矛盾加剧的直接调整）"
    ]
}


'在长征时期，中国共产党针对国内外形势的急剧变化和民族矛盾的显著加剧，进行了重要的方针政策调整。请仔细分析以下选项，并选出最符合这一历史事实的方针政策调整。'

In [16]:
def prompt_eval_agent(prompt):
    input_prompt = prompt_analyst
    response = llm(input_prompt + "/n 你要评分的prompt：" + prompt)
    return response

In [ ]:
print(prompt_eval_agent(p))

In [17]:
#original without optimization prompt
total_q = 40
count = 0
for i in range(total_q):
    initial_p, question_only, options= initial_prompt(i)
    response = chat_comp.do(
            model="ERNIE-Bot", 
            messages=[{
                "role": "user",
                "content": initial_p
                }],
            temperature = 0.00001)# ask agent respond to the question
    count += test(response.body['result'], i)

count / total_q

KeyboardInterrupt: 

In [28]:
count = 0
total_q = 40 #data.shape[0]
cot_prompt = """
######
请注意， 你的回答应该清晰、准确、简洁，并提供足够的细节来支持你的答案。让我们一步一步来思考。"""

for i in range(total_q):
    initial_p, question_only, options= initial_prompt(i)
    optimized_prompt = prompt_optimization(initial_p, options)
    prompt = f"""{initial_instruction} 
======
{prompt_preprocess(optimized_prompt)} 
======
{options}  
{cot_prompt}""" # generate new prompt
    print(f"new prompt is {prompt}")
    response = chat_comp.do(
            model="ERNIE-Bot", 
            messages=[{
                "role": "user",
                "content": prompt+cot_prompt
                }],
            temperature = 0.00001)# ask agent respond to the question
    count += test(response.body['result'], i)

count / total_q

{
    "提示词": "对此，下列历史解释正确的是",
    "优化后的提示词": "请从以下选项中，选择一个最能准确解释工商业者在中国近代社会发展中作用和贡献的答案。",
    "选项": [
        "A. 工商业者是倡导革命的新中坚力量",
        "B. 工商业者依旧属于传统的社会群体",
        "C. 工商业者投身社会事务是有利可图",
        "D. 工商业者推动了近代中国社会发展"
    ]
}
new prompt is 
这个题目是单选题

#回答格式#
答案是【你的答案】来标注你的答案。

#答案选项#
A

B

C

D

 
请从以下选项中，选择一个最能准确解释工商业者在中国近代社会发展中作用和贡献的答案。 
A 工商业者是倡导革命的新中坚力量 
 B 工商业者依旧属于传统的社会群体 
 C 工商业者投身社会事务是有利可图 
 D 工商业者推动了近代中国社会发展  

######
请注意， 你的回答应该清晰、准确、简洁，并提供足够的细节来支持你的答案。让我们一步一步来思考。
文言一心的回答是：答案是【D】工商业者推动了近代中国社会发展。

这个选项最准确地概括了工商业者在中国近代社会发展中的核心作用和贡献。工商业者通过发展工业和商业，促进了经济的增长和市场的繁荣，为近代中国社会的转型和现代化进程提供了重要的动力。他们不仅创造了大量的物质财富，还推动了技术革新、产业升级和社会结构的变革。相比之下，A选项虽然强调了工商业者的某种政治作用，但并非其主要贡献；B选项则明显与事实不符，因为工商业者在中国近代社会中已经逐渐脱离了传统的社会群体，成为推动社会进步的重要力量；C选项虽然可能在一定程度上反映了部分工商业者的动机，但并非其整体作用和贡献的准确描述。因此，D选项是最能准确解释工商业者在中国近代社会发展中作用和贡献的答案。
Answer is D
question 0 is correct!
{
    "提示词": "乾隆时江南地主土地管理特点分析选择题",
    "优化后的提示词": "请根据乾隆时期江南地区地主'所居在城或他州异县，地亩山场皆委之佃户'以及苏州'土著安业者田不满百亩，余皆佃农也。上田半归于郡城之富户'的史料，分析并判断以下哪个选项最准确地反映了当时江南地区的农业生产和社会经济结构

0.65